# GLLVM Simulations

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from gllvm import GLLVM, PoissonGLM, BinomialGLM, GaussianGLM


seed = 123
device = "cuda"

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# ----------------------------------------------------------
# 1. Generate synthetic data from the ground-truth model
# ----------------------------------------------------------

gllvm0 = GLLVM(latent_dim=1, output_dim=5)
gllvm0.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm0.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm0.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

num_samples = 10000

with torch.no_grad():
    gllvm0.wz.mul_(1.0)
    z0 = gllvm0.sample_z(num_samples)
    y0 = gllvm0.sample(z=z0)

# ----------------------------------------------------------
# 2. Build a fresh model that will be trained with VI
# ----------------------------------------------------------

gllvm = GLLVM(latent_dim=1, output_dim=5)
gllvm.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

# ----------------------------------------------------------
# 3. Build the encoder q(z|y)
# ----------------------------------------------------------

class Encoder(nn.Module):
    def __init__(self, input_dim=5, latent_dim=1, hidden=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )
        self.mean = nn.Linear(hidden, latent_dim)
        self.logvar = nn.Linear(hidden, latent_dim)

    def forward(self, y):
        h = self.net(y)
        mu = self.mean(h)
        logvar = self.logvar(h)
        return mu, logvar

encoder = Encoder(input_dim=5, latent_dim=1)
optimizer = optim.Adam(list(gllvm.parameters()) + list(encoder.parameters()), lr=1e-3)

# ----------------------------------------------------------
# 4. VI training loop: Standard VAE
# ----------------------------------------------------------

gllvm.to(device)
gllvm0.to(device)
encoder.to(device)
y0 = y0.to(device)


batch_size = 1000
num_epochs = 200

dataset = y0
n = len(dataset)

for epoch in range(num_epochs):
    perm = torch.randperm(n)

    total_elbo = 0.0
    for i in range(0, n, batch_size):
        idx = perm[i:i+batch_size].to(device)
        y = dataset[idx]

        # Encode q(z|y)
        mu, logvar = encoder(y)
        std = torch.exp(0.5 * logvar)

        # Reparameterization
        eps = torch.randn_like(std)
        z = mu + eps * std

        # Decoder log-likelihood
        logpy_z = gllvm.log_prob(y, z=z).sum(dim=-1)  # sum over response dims

        # KL(q||p)
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)

        elbo = (logpy_z - kl).mean()
        loss = -elbo

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_elbo += elbo.item()

    print(f"Epoch {epoch+1}: ELBO={total_elbo:.2f}")



Epoch 1: ELBO=-27.45
Epoch 2: ELBO=-26.30
Epoch 3: ELBO=-26.43
Epoch 4: ELBO=-25.88
Epoch 5: ELBO=-25.49
Epoch 6: ELBO=-25.37
Epoch 7: ELBO=-24.60
Epoch 8: ELBO=-24.70
Epoch 9: ELBO=-24.29
Epoch 10: ELBO=-24.48
Epoch 11: ELBO=-23.83
Epoch 12: ELBO=-22.95
Epoch 13: ELBO=-22.75
Epoch 14: ELBO=-22.25
Epoch 15: ELBO=-22.33
Epoch 16: ELBO=-21.77
Epoch 17: ELBO=-21.08
Epoch 18: ELBO=-21.31
Epoch 19: ELBO=-20.91
Epoch 20: ELBO=-20.82
Epoch 21: ELBO=-20.35
Epoch 22: ELBO=-20.05
Epoch 23: ELBO=-19.65
Epoch 24: ELBO=-19.25
Epoch 25: ELBO=-19.37
Epoch 26: ELBO=-18.88
Epoch 27: ELBO=-18.44
Epoch 28: ELBO=-18.20
Epoch 29: ELBO=-17.86
Epoch 30: ELBO=-17.86
Epoch 31: ELBO=-17.61
Epoch 32: ELBO=-17.37
Epoch 33: ELBO=-17.03
Epoch 34: ELBO=-16.91
Epoch 35: ELBO=-16.61
Epoch 36: ELBO=-16.27
Epoch 37: ELBO=-16.43
Epoch 38: ELBO=-16.15
Epoch 39: ELBO=-16.10
Epoch 40: ELBO=-15.92
Epoch 41: ELBO=-15.92
Epoch 42: ELBO=-15.57
Epoch 43: ELBO=-15.65
Epoch 44: ELBO=-15.40
Epoch 45: ELBO=-15.46
Epoch 46: ELBO=-15.

In [ ]:
with torch.no_grad():
    print(torch.mean((gllvm0.wz - gllvm.wz)**2), torch.mean((gllvm0.wz + gllvm.wz)**2))

tensor(15.5199, device='cuda:0')


In [8]:
gllvm0.wz *-1

tensor([[ 0.5573, -0.6018,  1.8482,  1.2021,  5.9846]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [9]:
gllvm.wz

Parameter containing:
tensor([[ 0.2540, -0.1652,  0.4313,  0.5598,  2.2648]], device='cuda:0',
       requires_grad=True)

In [10]:
print("True W_z:\n", gllvm0.wz)
print("Estimated W_z:\n", gllvm.wz)

print("True bias:\n", gllvm0.bias)
print("Estimated bias:\n", gllvm.bias)

print("True scale:\n", gllvm0.scale)
print("Estimated scale:\n", gllvm.scale)


True W_z:
 Parameter containing:
tensor([[-0.5573,  0.6018, -1.8482, -1.2021, -5.9846]], device='cuda:0',
       requires_grad=True)
Estimated W_z:
 Parameter containing:
tensor([[ 0.2540, -0.1652,  0.4313,  0.5598,  2.2648]], device='cuda:0',
       requires_grad=True)
True bias:
 Parameter containing:
tensor([0., 0., 0., 0., 0.], device='cuda:0', requires_grad=True)
Estimated bias:
 Parameter containing:
tensor([-5.1847e-03,  1.7326e-01, -4.4313e-02, -1.6309e-04, -1.4353e-01],
       device='cuda:0', requires_grad=True)
True scale:
 tensor([1., 1., 1., 1., 1.], device='cuda:0', grad_fn=<ExpBackward0>)
Estimated scale:
 tensor([1.0000, 1.0000, 1.0000, 0.8848, 1.3216], device='cuda:0',
       grad_fn=<ExpBackward0>)
